In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./all.csv')

In [3]:
df.head()

,code,text
0,Positive Activating,I found another interesting clue in your first...
1,Personal advice,It is good to see more details on this.
2,Vocatives (addressing individual),"Hi Daniel,Thanks for your comments."
3,Positive Deactivating,This is cool!This confirms my personal impress...
4,"Complementing, expressing appreciation",The final result is quite compelling and reall...


In [4]:
df.groupby('code')['text'].count()

code
Addresses or refers to the group using inclusive pronouns (addressing the whole group)     15
Asking questions                                                                          180
Complementing, expressing appreciation                                                    371
Course reflection                                                                           2
Expressing agreement                                                                       79
Expressing disagreement                                                                    28
Group cohesion                                                                              1
Negative Activating                                                                       157
Negative Deactivating                                                                      67
Open communication                                                                          2
Personal advice                                        

In [5]:
# over_50_cols = [
#     'Asking questions', 'Complementing, expressing appreciation', 'Expressing agreement',
#     'Negative Activating', 'Negative Deactivating', 'Personal advice',
#     'Phatics, salutations and greetings, Social sharing', 'Positive Activating',
#     'Self-disclosure & expressing values', 'Unconventional emotion expression', 'Vocatives (addressing individual)'
# ]
over_100_cols = [
    'Asking questions', 'Complementing, expressing appreciation',
    'Negative Activating', 'Personal advice',
    'Vocatives (addressing individual)'
]
df_over_50 = df[df['code'].isin(over_100_cols)]
df_over_50.groupby('code')['text'].count()

code
Asking questions                          180
Complementing, expressing appreciation    371
Negative Activating                       157
Personal advice                           202
Vocatives (addressing individual)         183
Name: text, dtype: int64

In [212]:
df_cleaned.to_csv('./filtered.csv')

In [6]:
import re
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
    df[text_field] = df[text_field].str.replace(r"@", "at")
    
    df[text_field] = df[text_field].str.lower()
    return df

In [ ]:
df_cleaned = standardize_text(df_over_50, 'text')
df_cleaned.head()

In [136]:
df_cleaned[df_cleaned['code'] == 'Asking questions']

,code,text,tokens
6,Asking questions,did you find any similarities about these two ...,"[did, you, find, any, similarities, about, the..."
15,Asking questions,it would interesting to see how well this data...,"[it, would, interesting, to, see, how, well, t..."
16,Asking questions,in which year does the population data start i...,"[in, which, year, does, the, population, data,..."
33,Asking questions,were they using a different data set?,"[were, they, using, a, different, data, set]"
37,Asking questions,does this mean that there is no data for the g...,"[does, this, mean, that, there, is, no, data, ..."
41,Asking questions,what is the large red circle in florida? why i...,"[what, is, the, large, red, circle, in, florid..."
42,Asking questions,does attending school directly make one literate?,"[does, attending, school, directly, make, one,..."
48,Asking questions,was uncertainty (prediction confidence interva...,"[was, uncertainty, prediction, confidence, int..."
54,Asking questions,have you thought of other data or known driver...,"[have, you, thought, of, other, data, or, know..."
82,Asking questions,"however, i don t understand where the focus on...","[however, i, don, t, understand, where, the, f..."


In [9]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

df_cleaned["tokens"] = df_cleaned["text"].apply(tokenizer.tokenize)

/Users/cl38824/anaconda2/envs/stan/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [10]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

all_words = [word for tokens in df_cleaned["tokens"] for word in tokens]
sentence_lengths = [len(tokens) for tokens in df_cleaned["tokens"]]
VOCAB = sorted(list(set(all_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_words), len(VOCAB)))
print("Max sentence length is %s" % max(sentence_lengths))

Using TensorFlow backend.


13386 words total, with a vocabulary size of 2135
Max sentence length is 55


In [11]:
import gensim

word2vec_path = "GoogleNews-vectors-negative300.bin.gz"
word2vec = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [12]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list) < 1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_questions, generate_missing=False):
    embeddings = clean_questions['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

In [13]:
embeddings = get_word2vec_embeddings(word2vec, df_cleaned)

## Attention Layer

In [15]:
import keras as K
from keras.layers import *

def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    

class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

## RNN

In [56]:
EMBEDDING_DIM = 300
MAX_SEQUENCE_LENGTH = max(sentence_lengths)
VOCAB_SIZE = len(VOCAB)

VALIDATION_SPLIT=.2
tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(df_cleaned["text"].tolist())
sequences = tokenizer.texts_to_sequences(df_cleaned["text"].tolist())

rnn_data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

factorized_codes = pd.factorize(df_cleaned["code"])[0]
factorized_codes_dict = pd.factorize(df_cleaned["code"])[1]

labels = factorized_codes

indices = np.arange(rnn_data.shape[0])
np.random.shuffle(indices)
rnn_data = rnn_data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * rnn_data.shape[0])

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

embedding_weights = np.zeros((len(word_index)+1, EMBEDDING_DIM))
for word, index in word_index.items():
    embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(embedding_weights.shape)

Found 2135 unique tokens.
(2136, 300)


In [90]:
from keras.models import Sequential
from keras.layers import *
from keras.preprocessing.sequence import pad_sequences

def rnn_text(embeddings, max_sequence_length, num_words, embedding_dim, labels_index): 
    model = Sequential()
    model.add(Embedding(num_words,
                        embedding_dim,
                        weights=[embeddings],
                        input_length=max_sequence_length,
                        mask_zero=True,
                        trainable=False))
    # model.add(Bidirectional(LSTM(100)))
    model.add(LSTM(128, return_sequences=True))
    model.add(LSTM(64, return_sequences=True))
    model.add(AttentionWithContext())
    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.1))
    model.add(Dense(64, activation="relu"))
    model.add(Dropout(0.1))
    model.add(Dense(32, activation="relu"))
    model.add(Dense(labels_index, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [91]:
x_train = rnn_data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = rnn_data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

rnn_model = rnn_text(
    embedding_weights, 
    MAX_SEQUENCE_LENGTH, 
    len(word_index)+1, 
    EMBEDDING_DIM, 
    len(list(df_cleaned["code"].unique()))
)

In [151]:
from keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)
rnn_fit = rnn_model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=100, batch_size=64, callbacks=[es])

Train on 875 samples, validate on 218 samples
Epoch 1/100
875/875 [==============================] - 2s 3ms/step - loss: 0.1126 - acc: 0.9166 - val_loss: 3.0942 - val_acc: 0.6651
Epoch 2/100
875/875 [==============================] - 2s 3ms/step - loss: 0.1123 - acc: 0.9143 - val_loss: 3.0737 - val_acc: 0.6697
Epoch 3/100
875/875 [==============================] - 2s 3ms/step - loss: 0.1141 - acc: 0.9097 - val_loss: 3.0120 - val_acc: 0.6835
Epoch 4/100
875/875 [==============================] - 2s 3ms/step - loss: 0.1124 - acc: 0.9120 - val_loss: 2.9247 - val_acc: 0.6743
Epoch 5/100
875/875 [==============================] - 2s 3ms/step - loss: 0.1121 - acc: 0.9200 - val_loss: 2.9188 - val_acc: 0.6789
Epoch 6/100
875/875 [==============================] - 2s 2ms/step - loss: 0.1122 - acc: 0.9223 - val_loss: 2.9265 - val_acc: 0.6743
Epoch 7/100
875/875 [==============================] - 2s 2ms/step - loss: 0.1123 - acc: 0.9189 - val_loss: 2.9398 - val_acc: 0.6743
Epoch 8/100
875/875 [==

In [159]:
def predict_text(model, text):
        predict = model.predict(pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=MAX_SEQUENCE_LENGTH)).ravel()
        return sorted([{"code": factorized_codes_dict[idx], "prob": prob} for idx, prob in enumerate(predict)], reverse=True, key=lambda x: x['prob'])
    
predict_text(rnn_model, "I don't like this feeling")

[{'code': 'Complementing, expressing appreciation', 'prob': 0.9998149},
 {'code': 'Negative Activating', 'prob': 0.00015111157},
 {'code': 'Personal advice', 'prob': 3.071309e-05},
 {'code': 'Vocatives (addressing individual)', 'prob': 3.2058363e-06},
 {'code': 'Asking questions', 'prob': 2.707606e-08}]

In [163]:
rnn_model.save('./rnn.h5')

In [161]:
import pydot as pyd
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import keras

keras.utils.vis_utils.pydot = pyd

SVG(model_to_dot(rnn_model).create(prog='dot', format='svg'))

218/218 [==============================] - 0s 1ms/step


[3.0567792052522713, 0.674311926058673]

## CNN

In [42]:
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import Concatenate
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.layers import LSTM, Bidirectional
from keras.models import Model

def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index, trainable=False, extra_conv=True):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=trainable)

    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    # Yoon Kim model (https://arxiv.org/abs/1408.5882)
    convs = []
    filter_sizes = [3,4,5]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=128, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = MaxPooling1D(pool_size=3)(l_conv)
        convs.append(l_pool)

    l_merge = Concatenate(axis=1)(convs)

    # add a 1D convnet with global maxpooling, instead of Yoon Kim model
    conv = Conv1D(filters=128, kernel_size=3, activation='relu')(embedded_sequences)
    pool = MaxPooling1D(pool_size=3)(conv)

    if extra_conv==True:
        x = Dropout(0.5)(l_merge)  
    else:
        # Original Yoon Kim model
        x = Dropout(0.5)(pool)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    #x = Dropout(0.5)(x)

    preds = Dense(labels_index, activation='softmax')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])

    return model

In [43]:
factorized_codes = pd.factorize(df_cleaned["code"])[0]
factorized_codes_dict = pd.factorize(df_cleaned["code"])[1]

cnn_labels = to_categorical(np.asarray(factorized_codes))

indices = np.arange(rnn_data.shape[0])
np.random.shuffle(indices)
rnn_data = rnn_data[indices]
cnn_labels = cnn_labels[indices]

x_train = rnn_data[:-num_validation_samples]
y_train = cnn_labels[:-num_validation_samples]
x_val = rnn_data[-num_validation_samples:]
y_val = cnn_labels[-num_validation_samples:]

cnn_model = ConvNet(embedding_weights, MAX_SEQUENCE_LENGTH, len(word_index)+1, EMBEDDING_DIM, 
                len(list(df_cleaned["code"].unique())), False)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
cnn_model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=100, batch_size=64)

In [155]:
from keras.utils import print_summary, plot_model

print_summary(cnn_model)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 55)           0                                            
__________________________________________________________________________________________________
embedding_26 (Embedding)        (None, 55, 300)      640800      input_12[0][0]                   
__________________________________________________________________________________________________
conv1d_36 (Conv1D)              (None, 53, 128)      115328      embedding_26[0][0]               
__________________________________________________________________________________________________
conv1d_37 (Conv1D)              (None, 52, 128)      153728      embedding_26[0][0]               
__________________________________________________________________________________________________
conv1d_38 

In [200]:
cnn_labels.shape

(1093, 5)

In [61]:
tokenizer.texts_to_sequences(["thank you very much!"])

[[24, 4, 21, 93]]